To start, define the user-specific constants:

In [ ]:
'''
These parameters describe the FRB that you want results for. Example:

FRB_Name = 'Spock'
FRB_Ra = 323.477
FRB_Dec = 13.244
host_redshift = 0.367
'''


'''
Put your Casjobs user ID here. If you do not have one, you can make an account at: https://mastweb.stsci.edu/ps1casjobs/CreateAccount.aspx
'''

CASJOBS_WSID_string = '' #string

Now let's import dependencies and input the FRB information:

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.table import Row
from astropy.io import fits
from astropy.utils.data import download_file
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
from astropy.cosmology import WMAP9 as cosmo
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('TkAgg')
from numpy import *
from pylab import *
from scipy import stats
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import coordinates, units
from astropy.io import ascii
import mastcasjobs
from psquery import mastquery, sed
import dustmaps.sfd
import getpass
import os
os.environ["CASJOBS_WSID"] = CASJOBS_WSID_string
import re
from psquery import get_coord
import time, sys, os
import h5py
import numpy as np
import scipy
import fsps
import sedpy
import prospect
import emcee
import dynesty
from matplotlib.pyplot import *

%matplotlib inline

# re-defining plotting defaults
from matplotlib.font_manager import FontProperties
from matplotlib import gridspec

In [ ]:
sizestring = '2400'
size = 2400
radius = 15
cone = 300/3600 #in degrees
format = 'fits'

Let's set up our output files.

In [ ]:
file = open(FRB_Name+' Results.txt', 'w')
file.write(FRB_Name+' coordinates: ('+str(FRB_Ra)+', '+str(FRB_Dec)+')\n\n')
file.close()

Now, let's define the file reading functions:

In [ ]:
def geturl(ra, dec, size, filter):

    '''
    Function that creates a PanStarrs FITS url to download.

    Parameters:
    -----------
    ra  : float
        J2000 RA of center of image in degrees

    dec : float
        J2000 Dec of center of image in degrees

    size : integer
        image size in pixels where 240 pixels is 60 arcsec

    filter : string
        desired filter for the image (g, r, i, z, or y)

    Returns:
    --------
    url : string
        desired url to unpack for a FITS file
    '''
    
    format, output_size, color = "fits", None, False

    service = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"
    
    url = ("{service}?ra={ra}&dec={dec}&size={size}&format=fits"
           "&filters={filter}").format(**locals())
    table = Table.read(url, format='ascii')
    
    url = ("https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
           "ra={ra}&dec={dec}&size={size}&format={format}").format(**locals())

    urlbase = url + "&red="
    url = []
    for filename in table['filename']:
        url.append(urlbase+filename)
        
    return url

def readfile(ra, dec, size, filter):
      '''
      This function runs the get_url() function, creates a path, and downloads the FITS file data and header. 

      Parameters:
      -----------
      ra  : float
            J2000 RA of center of image in degrees

      dec : float
            J2000 Dec of center of image in degrees

            size : integer
            image size in pixels where 240 pixels is 60 arcsec

      filter : string
                  desired filter for the image (g, r, i, z, or y)

      Returns:
      -------
      data : numpy.ndarray
            data describing the FITS file

      header: astropy.io.fits.header.Header
              header for the FITS file containing information about the collection processes and image reduction
      '''
      if filter == 'h' or filter=='j':
            path = filter+'_STACK_'+FRB_Name.upper()+'.fits'
      else:
            url=geturl(ra, dec, size, filter)[0]
            path = download_file(url)

      hdu = fits.open(path)
      data = hdu[0].data
      header = hdu[0].header
      hdu.close()

      return [data, header]


We're going to run a query in the STRM database to pull surrounding galaxies:

In [ ]:
'''
This cell uses the mastquery function from Casey Law's psquery package.

Parameters:
-----------

FRB_Ra : float
         RA defined above, in degrees

FRB_Dec : float
          Dec defined above, in degrees

cone : float
       radius of cone search defined above, in degrees

selectcol : list
            desired information from STRM, out of ["objID", "raMean", "decMean", "z_phot0", "z_photErr", "prob_Galaxy", "prob_Star", "prob_QSO"]

Returns:
--------

STRM_results : astropy table
               table of results from the mastquery search

intervening_coords : list
                     this is the list created in the STRM mastquery cell
                     it has entries with the following form: 
                     [STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr']]

'''

deg_in_rad = 57.2958
virial_radius_mpc = 0.3

STRM_results = mastquery.cone_ps1strm((FRB_Ra, FRB_Dec), cone, selectcol=["raMean", "decMean", "z_phot0", "z_photErr", "prob_Galaxy", "prob_Star"])
intervening_coords = []
i = 0
while i < len(STRM_results):
    FRB_coord = SkyCoord(FRB_Ra*units.deg, FRB_Dec*units.deg, frame='icrs')
    temp_coord = SkyCoord(STRM_results[i]['raMean']*units.deg, STRM_results[i]['decMean']*units.deg, frame='icrs')
    separation = (FRB_coord.separation(temp_coord)).deg
    d_mpc = (cosmo.angular_diameter_distance(z=(STRM_results[i]['z_phot0'])-2*STRM_results[i]['z_photErr'])).to_value()
    theta_virial = 57.2958*(0.3/d_mpc)
    #let's get rid of objects that probably aren't galaxies
    if STRM_results[i]['prob_Galaxy'] < 0.5:
        STRM_results.remove_row(i)
    elif STRM_results[i]['prob_Star'] > 0.5:
        STRM_results.remove_row(i)
    #any object with a separation larger than the largest Virial radius is not at risk of intervening
    elif separation > theta_virial:
        STRM_results.remove_row(i)
    #any object more distant than the FRB host is not at risk of intervening
    #we use the smallest possible extreme of the zphot
    elif STRM_results[i]['z_phot0'] - 2*STRM_results[i]['z_photErr'] > host_redshift:
        STRM_results.remove_row(i)
    #we assume that any object without a real zphot detection is not at risk of intervening
    elif STRM_results[i]['z_phot0'] < 0:
        STRM_results.remove_row(i)
    else:
        intervening_coords.append([STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr'], separation])
        i+=1

This function runs the aperture photometry functions from photutils and calculates the error. 

The magnitude is found by the following equation:
$$
M = M_{ZPT} - 2.5\log_{10}{\frac{c \cdot g}{t_{exp}}} 
$$
...where $M_{ZPT}$ is the zero-point magnitude (25 for PanStarrs FITS files), $t_{exp}$ is the exposure time in seconds, $c$ is the counts per pixel, and $g$ is the photon-electron gain.

The error in this magnitude can be approximated by:
$$
\sigma_{M} \approx \frac{-2.5}{\ln{10} \cdot c} \sqrt{\left(A_{aperture} + \frac{A_{aperture}^2}{A_{BG}}\right)\sigma_{BG}^2}
$$

And the 3-sigma upper limit on non-detections is then found with:

$$
M_{3\sigma} = M_{ZPT} - 2.5\log{\left[\mu_{BG}+3\sigma_{BG}*\frac{g}{t_{exp}}\right]}\pm{\left|\frac{2.5}{\ln{10} \cdot c}\right|\sqrt{\sigma_{BG}^2\left(A_{aperture}+\frac{A_{aperture}^2}{A_{BG}}\right)}}
$$

In [ ]:
def getmagnitude(data, header, radius, position, filter):

    '''
    Function that handles the aperture photometry and calculates associated error. 

    Parameters:
    ----------
    data : numpy.ndarray
           data describing the FITS file

    header : astropy.io.fits.header.Header
             header for the FITS file containing information about the collection processes and image reduction

    radius : integer
             radius of aperture in pixels

    position : list 
               contains x and y coordinates (in pixels) of the center of the aperture

    Returns:
    --------
    mag_and_error : list
                    contains either the magnitude and error or the 3-sigma upper limit and the error
                    if there is no detection, the error will be negative
    
    '''

    Texp = header['EXPTIME']

    if filter == 'h' or filter=='j':
        ZPTmag = float(header['TMC_ZP'])
        gain = float(header['GAIN'])
    else:
        ZPTmag = header['HIERARCH FPA.ZP']
        gain = header['HIERARCH CELL.GAIN']

    aperture = CircularAperture(position, radius)
    annulus = CircularAnnulus(position, radius+2, np.sqrt(2*(radius**2+2*radius+2))) 

    phot_table = aperture_photometry(data, aperture)
    ann_table = aperture_photometry(data, annulus)

    counts = phot_table['aperture_sum'] - (ann_table['aperture_sum'])*(aperture.area/annulus.area)
    
    annstats  = ApertureStats(data, annulus)
    annvar = (annstats.std)**2

    count_error = np.sqrt(annvar*(aperture.area+(aperture.area**2)/annulus.area))
    error = (2.5/(np.log(10)*counts))*count_error
    upperlimcounts = annstats.mean+3*annstats.std
    Ne = counts*gain
    
    if Ne/Texp > 0:
        magnitude = ZPTmag - 2.5*np.log10(Ne/Texp)
        mag_and_error = [magnitude[0], error[0]] 
    else:
        Ne = ann_table['aperture_sum']*gain
        magnitude = ('3σ: '+str(ZPTmag - 2.5*np.log10(upperlimcounts*gain/Texp)))
        mag_and_error = [magnitude, str("{:.4f}".format(error[0]))]

    return mag_and_error

In [ ]:
def getinfo(FRB_Name, FRBra, FRBdec, intervening_coords):

    '''
    Function that runs the get_magnitude function for a number of coordinates in the same field of view. 

    Parameters:
    -----------

    FRB_Name : string
               name of FRB defined above

    FRB_Ra : float
         RA defined above, in degrees

    FRB_Dec : float
          Dec defined above, in degrees

    intervening_coords : list
                         this is the list created in the STRM mastquery cell
                         it has entries with the following form: 
                         [STRM_results[i]['raMean'], STRM_results[i]['decMean'], STRM_results[i]['z_phot0'], STRM_results[i]['z_photErr']]

    Returns: 
    --------

    df : Pandas DataFrame
         this contains all of the aperture photometry detections in each bandpass for each candidate
    
    '''
    Name, Ra, Dec, Separation, zphot, zphot_err = [], [], [], [], [], []
    filterdict = {'g':[], 'r':[], 'i':[], 'z':[], 'y':[], 'h':[], 'j':[]}
    errordict = {'err_g':[], 'err_r':[], 'err_i':[], 'err_z':[], 'err_y':[], 'err_h':[], 'err_j':[]}
    
    j = 0
    for coord in intervening_coords:
        Name.append(FRB_Name+str(j))
        ratemp = coord[0]
        dectemp = coord[1]
        zphot_temp = coord[2]
        zphot_err_temp = coord[3]
        separation_temp = coord[4]
        Ra.append(ratemp)
        Dec.append(dectemp)
        zphot.append(zphot_temp)
        zphot_err.append(zphot_err_temp)
        Separation.append(separation_temp)
        j+=1

    for filter in filterdict:
        file = readfile(FRBra, FRBdec, size, filter)
        data = file[0]
        header = file[1]
        w = WCS(header)

        for coord in intervening_coords:
            ratemp = coord[0]
            dectemp = coord[1]  
            sky = SkyCoord(ra = ratemp, dec = dectemp, unit='deg')
            position = w.world_to_pixel(sky)
            mag_and_error = getmagnitude(data, header, radius, position, filter)
            filterdict[filter].append(mag_and_error[0])
            errordict['err_'+filter].append(mag_and_error[1])


    content = {'Name': Name, 'RA': Ra, 'Dec': Dec, 'Separation': Separation, 'zphot': zphot, 'zphot_err': zphot_err,
               'g': filterdict['g'], 'err_g' : errordict['err_g'],
               'r': filterdict['r'], 'err_r' : errordict['err_r'],
               'i': filterdict['i'], 'err_i' : errordict['err_i'],
               'z': filterdict['z'], 'err_z' : errordict['err_z'],
               'y': filterdict['y'], 'err_y' : errordict['err_y'],
               'h': filterdict['h'], 'err_h' : errordict['err_h'],
               'j': filterdict['j'], 'err_j' : errordict['err_j']}


    df = pd.DataFrame(content)

    return df

Now we call a bunch of functions to perform aperture photometry on a wide range of candidates:

In [ ]:
intervening_info = {'Name':[],
                        'RA':[], 
                        'Dec':[], 
                        'Separation':[],
                        'g':[],
                        'err_g':[],
                        'r':[],
                        'err_r':[],
                        'i':[],
                        'err_i':[],
                        'z':[],
                        'err_z':[],
                        'y':[],
                        'err_y':[],
                        'h':[],
                        'err_h':[],
                        'j':[],
                        'err_j':[]}

df = getinfo(FRB_Name, FRB_Ra, FRB_Dec, intervening_coords)


Let's try to identify the detection that corresponds to the FRB host galaxy, and compare this to our known redshift.

In [ ]:
separations = df['Separation'].to_list()
lowest_separation = min(separations)
lowest_separation_index = np.argmin(separations)
lowest_zphot = df.at[lowest_separation_index, 'zphot']
lowest_zphot_err = df.at[lowest_separation_index, 'zphot_err']
lowest_RA = df.at[lowest_separation_index, 'RA']
lowest_Dec = df.at[lowest_separation_index, 'Dec']

print('Photometric redshift: '+str(lowest_zphot))
print('Photometric redshift error: '+str(lowest_zphot_err))
print('Spectroscopic redshift: '+str(host_redshift))

file = open(FRB_Name+' Results.txt', 'a')
file.write('Below is the information for the source listed in STRM with the smallest separation from the localized FRB.\n')
file.write('It is possible that STRM will not have usable data for this detection, that the object is not real, or that it is not the host galaxy.\n\n')
file.write('The closest detection was an object at ('+str(lowest_RA)+', '+str(lowest_Dec)+') with the following information:\n')
file.write('Photometric redshift: '+str(lowest_zphot))
file.write('\nPhotometric redshift: '+str(lowest_zphot_err))
file.write('\nCompare to the spectroscopic redshift: '+str(host_redshift))
file.close()

Now let's get rid of the incomplete detections: 

In [ ]:
idlist = ['Name', 'RA', 'Dec', 'Separation', 'zphot', 
'zphot_err', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i', 
'z', 'err_z', 'y', 'err_y', 'h', 'err_h', 'j', 'err_j']

detection_names = []
non_detection_names = []

detections = pd.DataFrame(columns = idlist)
non_detections = pd.DataFrame(columns = idlist)

for i in range(0, len(df)):
    for f in ['g', 'r', 'i', 'z', 'y', 'h', 'j']:
        #we place aside objects that might intervene but are not detected in all bandpasses
        if type(df.at[i,f]) == str:
            non_detection_names.append(FRB_Name+str(i))
            break
        elif df.at[i,f] > 25.0:
            non_detection_names.append(FRB_Name+str(i))
            break
        #we consider the rest to be viable, detected candidates
        elif f == 'h':
            detection_names.append(FRB_Name+str(i))
        else:
            continue

for name in detection_names:
    line = pd.DataFrame([df.iloc[int(name.lstrip(FRB_Name))].to_list()], columns = idlist)
    detections = pd.concat([detections, line])

for name in non_detection_names:
    line = pd.DataFrame([df.iloc[int(name.lstrip(FRB_Name))].to_list()], columns = idlist)
    non_detections = pd.concat([non_detections, line])

detections = detections.set_index(pd.Index(range(0, len(detections))))
non_detections = non_detections.set_index(pd.Index(range(0, len(non_detections))))

print(len(detections))
print(len(non_detections))
print(detections)

file = open(FRB_Name+' Results.txt', 'a')
file.write('\n\nThe following sources were detected in all bandpasses and might intersect the FRB line of sight:\n\n')
for id in idlist:
    file.write(id+'   ')
file.write('\n')
file.close()
detections.to_csv(FRB_Name+' Results.txt', header=None, index=None, sep=' ', mode='a', float_format='%.4f')
file = open(FRB_Name+' Results.txt', 'a')
file.write('\n\nThe following sources were not detected in all bandpasses but might still intersect the FRB line of sight:\n\n')
for id in idlist:
    file.write(id+'   ')
file.write('\n')
file.close()
non_detections.to_csv(FRB_Name+' Results.txt', header=None, index=None, sep=' ', mode='a', float_format = '%.4f')

In [ ]:
import dustmaps.sfd
dustmaps.sfd.fetch()

In [ ]:
import psquery
for index in range(0, 1):

    galaxy_name = detections.at[index, 'Name']
    galaxy_separation = detections.at[index, 'Separation']
    # galaxy_magnitudes = []
    # galaxy_errors = []

    # for filter in ['g', 'r', 'i', 'z', 'y']:
    #     galaxy_magnitudes.append(detections.at[index, filter])
    #     galaxy_errors.append(detections.at[index, 'err_'+filter])
    # galaxy_redshift = detections.at[index, 'zphot']

    phot = {'ps_g': detections.at[index, 'g'],
    'ps_r': detections.at[index, 'r'],
    'ps_i': detections.at[index, 'i'],
    'ps_z': detections.at[index, 'z'],
    'ps_y': detections.at[index, 'y'],
    'twomass_H': detections.at[index, 'h'],
    'ps_g_err': detections.at[index, 'err_g'],
    'ps_r_err': detections.at[index, 'err_r'],
    'ps_i_err': detections.at[index, 'err_i'],
    'ps_z_err': detections.at[index, 'err_z'],
    'ps_y_err': detections.at[index, 'err_y'],
    'twomass_H_err': detections.at[index, 'err_h'],
    'free_redshift': False,
    'z': detections.at[index, 'zphot']}

    coords = (SkyCoord(ra=detections.at[index, 'RA']*units.degree, dec=detections.at[index, 'Dec']*units.degree, frame='icrs')).to_string('hmsdms')
    phot_extinct = sed.extinct(coords, phot)
    output = psquery.sed.run_fit(phot_extinct)

    file = open(FRB_Name+' Results.txt', 'a')
    file.write('\n'+galaxy_name+'\n')
    file.write('\nSeparation: '+str(galaxy_separation)+'\n')
    file.write('\nMass: ')
    file.write(str(output[2][0]))
    file.write('\nlogzsol: ')
    file.write(str(output[2][1]))
    file.write('\ndust2: ')
    file.write(str(output[2][2]))
    file.write('\ntage: ')
    file.write(str(output[2][3]))
    file.write('\ntau: ')
    file.write(str(output[2][4]))
    file.close()

In [ ]:
def build_obs(
    phot,
    filternames,
    mag_err_clip=0.05,
    **extras
):
    """Build a dictionary of observational data.

    phot = table or dictionary with photometry. Should already be extinction corrected.
    Upper limits have flux=0 and "err" equal to the limit (in what units?).
    filternames is list of bands and must match name in sedpy.
    If None, names selected from the set available in sedpy.

    returns obs: A dictionary of observational data to use in the fit.
    """

    obs = {}
    if filternames is None:
        available = py.observate.list_available_filters()
        sel = lambda x: any([(st in x.lower()) for st in available if "err" not in x])
        filternames = list(filter(sel, phot.keys()))
    flt_use = np.array([], dtype="S20")
    data_use, edata_use = np.array([]), np.array([])
    for k in filternames:
        if phot[k] != 0:
            flt_use = np.append(flt_use, k)
            data_use = np.append(data_use, phot[k])
            edata_use = np.append(edata_use, phot[k + "_err"])

    edata_use = np.array(edata_use)
    edata_use = np.clip(edata_use, mag_err_clip, 10)
    obs["islim"] = edata_use <= 0

    obs["filternames"] = filternames
    obs["filters"] = sedpy.observate.load_filters(filternames)
    obs["mags"] = data_use
    obs["mags_unc"] = edata_use
    obs["maggies"] = 10 ** (-0.4 * obs["mags"])  # fluxes in units of maggies (Jy/3631)
    obs["maggies_unc"] = np.log(10) / 2.5 * obs["mags_unc"] * obs["maggies"]

    # deal with upper limits
    # assume upper limits are 5-sigma (true for GALEX)
    nsigma = np.repeat(5, len(obs["filternames"]))
    iswise = [i for i, w in enumerate(filternames) if "wise" in w]
    nsigma[iswise] = 2  # WISE limits are 95%CL

    obs["maggies_unc"][obs["islim"]] = (
        10 ** (-0.4 * obs["mags"][obs["islim"]]) / nsigma[obs["islim"]]
    )
    obs["maggies"][obs["islim"]] = 0

    # Photometry mask, True = include in fit
    obs["phot_mask"] = np.array([True for f in obs["filters"]])

    # This is an array of effective wavelengths for each of the filters.
    # It is not necessary, but it can be useful for plotting so we store it here as a convenience
    obs["phot_wave"] = np.array([f.wave_effective for f in obs["filters"]])

    # We do not have a spectrum, so we set some required elements of the obs dictionary to None.
    obs["wavelength"] = None
    obs["spectrum"] = None
    obs["unc"] = None
    obs["mask"] = None

    # This function ensures all required keys are present in the obs dictionary,
    # adding default values if necessary
    obs = fix_obs(obs)

    return obs

In [ ]:
print(phot)

In [ ]:
import os.path

try:
    import sedpy
    from prospect.fitting import fit_model, lnprobfn
    from prospect.likelihood import lnlike_phot, lnlike_spec, write_log
    from prospect.models import priors, sedmodel
    from prospect.models.templates import TemplateLibrary
    from prospect.sources import CSPSpecBasis
    from prospect.utils.obsutils import fix_obs
    from prospect.plotting import sfh
except ImportError:
    print("sedpy or prospect not importing. cannot use sed modeling...")
# new imports
from astropy.coordinates import SkyCoord
from astropy import cosmology
from astroquery.mast import Catalogs
from dustmaps.sfd import SFDQuery
from extinction import fitzpatrick99

from psquery import irsaquery, psquery, noaoquery

cosmo = cosmology.Planck18

phot = {'ps_g': detections.at[index, 'g'],
    'ps_r': detections.at[index, 'r'],
    'ps_i': detections.at[index, 'i'],
    'ps_z': detections.at[index, 'z'],
    'ps_y': detections.at[index, 'y'],
    'twomass_H': detections.at[index, 'h'],
    'ps_g_err': detections.at[index, 'err_g'],
    'ps_r_err': detections.at[index, 'err_r'],
    'ps_i_err': detections.at[index, 'err_i'],
    'ps_z_err': detections.at[index, 'err_z'],
    'ps_y_err': detections.at[index, 'err_y'],
    'twomass_H_err': detections.at[index, 'err_h'],

    'free_redshift': False,
    'z': detections.at[index, 'zphot']}

coords = (SkyCoord(ra=detections.at[index, 'RA']*units.degree, dec=detections.at[index, 'Dec']*units.degree, frame='icrs')).to_string('hmsdms')
phot_extinct = sed.extinct(coords, phot)

def build_obs(
    phot=None,
    filternames=None,
    mag_err_clip=0.05,
    standard=["galex", "ps", "sdss", "wise", "decam", "twomass"],
    **extras
):
    """Build a dictionary of observational data.
    phot = table or dictionary with photometry. Should already be extinction corrected.
    Upper limits have flux=0 and "err" equal to the limit (in what units?).
    filternames is list of bands and must match name in sedpy.
    If None, names inferred from phot using standard root names.
    returns obs: A dictionary of observational data to use in the fit.
    """

    obs = {}
    if filternames is None:
        sel = lambda x: any([(st in x.lower()) for st in standard if "err" not in x])
        filternames = list(filter(sel, phot.keys()))
    flt_use = np.array([], dtype="S20")
    data_use, edata_use = np.array([]), np.array([])
    for k in filternames:
        if phot[k] != 0:
            flt_use = np.append(flt_use, k)
            data_use = np.append(data_use, phot[k])
            edata_use = np.append(edata_use, phot[k + "_err"])

    edata_use = np.array(edata_use)
    edata_use = np.clip(edata_use, mag_err_clip, 10)
    obs["islim"] = edata_use <= 0

    obs["filternames"] = filternames
    obs["filters"] = sedpy.observate.load_filters(filternames)
    obs["mags"] = data_use
    obs["mags_unc"] = edata_use
    obs["maggies"] = 10 ** (-0.4 * obs["mags"])  # fluxes in units of maggies (Jy/3631)
    obs["maggies_unc"] = np.log(10) / 2.5 * obs["mags_unc"] * obs["maggies"]

    # deal with upper limits
    # assume upper limits are 5-sigma (true for GALEX)
    nsigma = np.repeat(5, len(obs["filternames"]))
    iswise = [i for i, w in enumerate(filternames) if "wise" in w]
    nsigma[iswise] = 2  # WISE limits are 95%CL

    obs["maggies_unc"][obs["islim"]] = (
        10 ** (-0.4 * obs["mags"][obs["islim"]]) / nsigma[obs["islim"]]
    )
    obs["maggies"][obs["islim"]] = 0

    # Photometry mask, True = include in fit
    obs["phot_mask"] = np.array([True for f in obs["filters"]])

    # This is an array of effective wavelengths for each of the filters.
    # It is not necessary, but it can be useful for plotting so we store it here as a convenience
    obs["phot_wave"] = np.array([f.wave_effective for f in obs["filters"]])

    # We do not have a spectrum, so we set some required elements of the obs dictionary to None.
    obs["wavelength"] = None
    obs["spectrum"] = None
    obs["unc"] = None
    obs["mask"] = None

    # This function ensures all required keys are present in the obs dictionary,
    # adding default values if necessary
    obs = fix_obs(obs)

    return obs
  
obs = build_obs(phot = phot_extinct)


In [ ]:
print(obs)